**This is the initial training script for inappropriate content detection, using YOLOv8 model.**

In [ ]:
%pip install torch torchvision tqdm

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from tqdm import tqdm

# Define YOLOv8 model
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.activation = nn.LeakyReLU(0.1)

    def forward(self, x):
        x = self.conv(x)
        x = self.batchnorm(x)
        x = self.activation(x)
        return x

class YOLOv8(nn.Module):
    def __init__(self, num_classes):
        super(YOLOv8, self).__init__()
        self.conv1 = ConvBlock(3, 64, 7, 2, 3)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv2 = ConvBlock(64, 192, 3, 1, 1)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv3_1 = ConvBlock(192, 128, 1, 1, 0)
        self.conv3_2 = ConvBlock(128, 256, 3, 1, 1)
        self.conv3_3 = ConvBlock(256, 256, 1, 1, 0)
        self.conv3_4 = ConvBlock(256, 512, 3, 1, 1)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.conv4_1 = ConvBlock(512, 256, 1, 1, 0)
        self.conv4_2 = ConvBlock(256, 512, 3, 1, 1)
        self.conv4_3 = ConvBlock(512, 256, 1, 1, 0)
        self.conv4_4 = ConvBlock(256, 512, 3, 1, 1)
        self.conv4_5 = ConvBlock(512, 256, 1, 1, 0)
        self.conv4_6 = ConvBlock(256, 512, 3, 1, 1)
        self.conv4_7 = ConvBlock(512, 256, 1, 1, 0)
        self.conv4_8 = ConvBlock(256, 512, 3, 1, 1)
        self.conv4_9 = ConvBlock(512, 512, 1, 1, 0)
        self.conv4_10 = ConvBlock(512, 1024, 3, 1, 1)
        self.pool4 = nn.MaxPool2d(2, 2)

        self.conv5_1 = ConvBlock(1024, 512, 1, 1, 0)
        self.conv5_2 = ConvBlock(512, 1024, 3, 1, 1)
        self.conv5_3 = ConvBlock(1024, 512, 1, 1, 0)
        self.conv5_4 = ConvBlock(512, 1024, 3, 1, 1)
        self.conv5_5 = ConvBlock(1024, 1024, 3, 1, 1)
        self.conv5_6 = ConvBlock(1024, 1024, 3, 2, 1)

        self.conv6_1 = ConvBlock(1024, 1024, 3, 1, 1)
        self.conv6_2 = ConvBlock(1024, 1024, 3, 1, 1)

        self.fc1 = nn.Linear(7 * 7 * 1024, 4096)
        self.fc2 = nn.Linear(4096, num_classes)

    def forward(self, x):
        x = self.pool1(self.conv1(x))
        x = self.pool2(self.conv2(x))
        x = self.pool3(self.conv3_4(self.conv3_3(self.conv3_2(self.conv3_1(x)))))
        x = self.pool4(self.conv4_10(self.conv4_9(self.conv4_8(self.conv4_7(self.conv4_6(self.conv4_5(self.conv4_4(self.conv4_3(self.conv4_2(self.conv4_1(x)))))))))))
        x = self.conv5_6(self.conv5_5(self.conv5_4(self.conv5_3(self.conv5_2(self.conv5_1(x))))))

        x = self.conv6_2(self.conv6_1(x))

        x = x.view(-1, 7 * 7 * 1024)
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x

# Define dataset class
class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = datasets.ImageFolder(root=data_dir, transform=transform)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img, label = self.data[idx]
        return img, label

In [ ]:

# Define transformations
transform = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
])

# Load dataset
dataset = CustomDataset(data_dir='\dataset', transform=transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define model, optimizer, and loss function
num_classes = 6
model = YOLOv8(num_classes=num_classes)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(dataloader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(dataloader)}")

In [ ]:
# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")
